In [3]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/2%20-%20Learning%20Phrase%20Representations%20using%20RNN%20Encoder-Decoder%20for%20Statistical%20Machine%20Translation.ipynb

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator
from nltk.translate.bleu_score import sentence_bleu


import spacy

import random
import math, time
import os


SEED = 1

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

train_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields=(SRC, TRG))

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")


print(vars(train_data.examples[0]))

SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (de) vocabulary: {len(TRG.vocab)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
{'src': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.'], 'trg': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'i', 'm', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']}
Unique tokens in source (en) vocabulary: 5923
Unique tokens in target (de) vocabulary: 7875


In [4]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)
print(len(train_iterator))
print(len(valid_iterator))
print(len(test_iterator))

cpu
227
8
8


In [5]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
        
        #outputs = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [6]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        # in case of encoder, first dimension was sent_len, but decoder is for each word, so sent_len becomes 1
        embed_new = torch.cat((embedded, context), dim=2)
        
        # adding at the last dimension
        output, hidden = self.rnn(embed_new, hidden)
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        output_new = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)
        # careful, first they have squeezed at dim = 0. since the first dimension was 1, it has now been removed. So, dim = 1
        # now corresponds to actual vector
        prediction = self.out(output_new)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [7]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        
        #src = [sent len, batch size]
        #trg = [sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
#         print("trg shape", trg.shape)
        trg_vocab_size = self.decoder.output_dim
#         print("batch size ", batch_size)
#         print("max len ", max_len)
#         print("trg vocab size ", trg_vocab_size)
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
#         print("shape of outputs")
#         print(outputs.shape)
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        context = self.encoder(src)
        hidden = context
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
#         print("first input")
#         print(input)
        for t in range(1, max_len):
#             print("in LOOP")
            output, hidden = self.decoder(input, hidden, context)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
#             print("output in loop")
#             print(output)
#             print(output.max(1))
            top1 = output.max(1)[1] # index of the max value
            input = (trg[t] if teacher_force else top1)
        
        return outputs

In [8]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters())

pad_idx = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)



In [9]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        print("train iterator ", i)
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        
        output = model(src, trg)
#         print("output after seq2seq")
#         print(output)
#         print(output.shape)
        #trg = [sent len, batch size]
        #output = [sent len, batch size, output dim]
        
        #reshape to:
        #trg = [(sent len - 1) * batch size]
        #output = [(sent len - 1) * batch size, output dim]
        x = output[1:].view(-1, output.shape[2])
        y = trg[1:].view(-1)
#         print("X ", x.shape)
#         print(x[0])
#         print("Y ", y.shape)
#         print(y)
        loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].view(-1))
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        
    return epoch_loss / len(iterator)

In [10]:
def process_final(t):
    sentences = []
    for i in range(len(t[0])):
        tensor_out = t[:, i]
        tensor_out = list(tensor_out.numpy())
        sentence = []
        for j in tensor_out:
            if TRG.vocab.itos[j] == '<pad>':
                break
            if TRG.vocab.itos[j] == '<eos>':
                sentence.append(TRG.vocab.itos[j])
                break
            sentence.append(TRG.vocab.itos[j])
        sentences.append(sentence)
    return sentences


def process_output(output):
    y = []
    for elem in output:
        x = []
        z = []
        for arr in elem:
            values, indices = arr.max(0)
            x.append(indices.item())
            z.append(values.item())
#         print("printing x")
#         print(x)
#         print("printing z")
#         print(z)


        y.append(x)
    final_tensor = torch.tensor(y)
    print("final tensor ")
    print(final_tensor)
    return process_final(final_tensor)





In [11]:
def evaluate(model, iterator, criterion, testing):
    
    model.eval()
    
    epoch_loss = 0
    bleu_score = 0
    count_pair = 0
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0) #turn off teacher forcing
            if testing:
                output_a = output[1:]
                trg_a = trg[1:]

                sent_out = process_output(output_a)
                sent_trg = process_final(trg_a)
                for o, t in zip(sent_out, sent_trg):
                    reference = [t]
                    candidate = o
                    print("reference ")
                    print(reference)
                    print("candidate ")
                    print(candidate)
                    bleu_score += sentence_bleu(reference, candidate)
                    count_pair += 1
  
            loss = criterion(output[1:].view(-1, output.shape[-1]), trg[1:].view(-1))

            epoch_loss += loss.item()
        if testing:
            print("count pair ", count_pair)
            print("bleu score ")


            print(bleu_score / count_pair)
    return epoch_loss / len(iterator)

In [12]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [13]:
N_EPOCHS = 10
CLIP = 1
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'seq2seq_improved_en_de.pt')


best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    print("epoch ", epoch)
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    print("train loss ", train_loss)
    valid_loss = evaluate(model, valid_iterator, criterion, False)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

epoch  0
train iterator  0
train iterator  1
train iterator  2
train iterator  3
train iterator  4
train iterator  5
train iterator  6
train iterator  7
train iterator  8
train iterator  9
train iterator  10
train iterator  11
train iterator  12
train iterator  13
train iterator  14
train iterator  15
train iterator  16
train iterator  17
train iterator  18
train iterator  19
train iterator  20
train iterator  21
train iterator  22
train iterator  23
train iterator  24
train iterator  25
train iterator  26
train iterator  27
train iterator  28
train iterator  29
train iterator  30
train iterator  31
train iterator  32
train iterator  33
train iterator  34
train iterator  35
train iterator  36
train iterator  37
train iterator  38
train iterator  39
train iterator  40
train iterator  41
train iterator  42
train iterator  43
train iterator  44
train iterator  45
train iterator  46
train iterator  47
train iterator  48
train iterator  49
train iterator  50
train iterator  51
train iterato

train iterator  186
train iterator  187
train iterator  188
train iterator  189
train iterator  190
train iterator  191
train iterator  192
train iterator  193
train iterator  194
train iterator  195
train iterator  196
train iterator  197
train iterator  198
train iterator  199
train iterator  200
train iterator  201
train iterator  202
train iterator  203
train iterator  204
train iterator  205
train iterator  206
train iterator  207
train iterator  208
train iterator  209
train iterator  210
train iterator  211
train iterator  212
train iterator  213
train iterator  214
train iterator  215
train iterator  216
train iterator  217
train iterator  218
train iterator  219
train iterator  220
train iterator  221
train iterator  222
train iterator  223
train iterator  224
train iterator  225
train iterator  226
train loss  3.62101810409109
Epoch: 02 | Time: 95m 31s
	Train Loss: 3.621 | Train PPL:  37.376
	 Val. Loss: 3.757 |  Val. PPL:  42.804
epoch  2
train iterator  0
train iterator  1


train iterator  139
train iterator  140
train iterator  141
train iterator  142
train iterator  143
train iterator  144
train iterator  145
train iterator  146
train iterator  147
train iterator  148
train iterator  149
train iterator  150
train iterator  151
train iterator  152
train iterator  153
train iterator  154
train iterator  155
train iterator  156
train iterator  157
train iterator  158
train iterator  159
train iterator  160
train iterator  161
train iterator  162
train iterator  163
train iterator  164
train iterator  165
train iterator  166
train iterator  167
train iterator  168
train iterator  169
train iterator  170
train iterator  171
train iterator  172
train iterator  173
train iterator  174
train iterator  175
train iterator  176
train iterator  177
train iterator  178
train iterator  179
train iterator  180
train iterator  181
train iterator  182
train iterator  183
train iterator  184
train iterator  185
train iterator  186
train iterator  187
train iterator  188


train iterator  91
train iterator  92
train iterator  93
train iterator  94
train iterator  95
train iterator  96
train iterator  97
train iterator  98
train iterator  99
train iterator  100
train iterator  101
train iterator  102
train iterator  103
train iterator  104
train iterator  105
train iterator  106
train iterator  107
train iterator  108
train iterator  109
train iterator  110
train iterator  111
train iterator  112
train iterator  113
train iterator  114
train iterator  115
train iterator  116
train iterator  117
train iterator  118
train iterator  119
train iterator  120
train iterator  121
train iterator  122
train iterator  123
train iterator  124
train iterator  125
train iterator  126
train iterator  127
train iterator  128
train iterator  129
train iterator  130
train iterator  131
train iterator  132
train iterator  133
train iterator  134
train iterator  135
train iterator  136
train iterator  137
train iterator  138
train iterator  139
train iterator  140
train ite

train iterator  41
train iterator  42
train iterator  43
train iterator  44
train iterator  45
train iterator  46
train iterator  47
train iterator  48
train iterator  49
train iterator  50
train iterator  51
train iterator  52
train iterator  53
train iterator  54
train iterator  55
train iterator  56
train iterator  57
train iterator  58
train iterator  59
train iterator  60
train iterator  61
train iterator  62
train iterator  63
train iterator  64
train iterator  65
train iterator  66
train iterator  67
train iterator  68
train iterator  69
train iterator  70
train iterator  71
train iterator  72
train iterator  73
train iterator  74
train iterator  75
train iterator  76
train iterator  77
train iterator  78
train iterator  79
train iterator  80
train iterator  81
train iterator  82
train iterator  83
train iterator  84
train iterator  85
train iterator  86
train iterator  87
train iterator  88
train iterator  89
train iterator  90
train iterator  91
train iterator  92
train iterat

train iterator  225
train iterator  226
train loss  1.9113804532567835
Epoch: 09 | Time: 48m 46s
	Train Loss: 1.911 | Train PPL:   6.762
	 Val. Loss: 3.472 |  Val. PPL:  32.216
epoch  9
train iterator  0
train iterator  1
train iterator  2
train iterator  3
train iterator  4
train iterator  5
train iterator  6
train iterator  7
train iterator  8
train iterator  9
train iterator  10
train iterator  11
train iterator  12
train iterator  13
train iterator  14
train iterator  15
train iterator  16
train iterator  17
train iterator  18
train iterator  19
train iterator  20
train iterator  21
train iterator  22
train iterator  23
train iterator  24
train iterator  25
train iterator  26
train iterator  27
train iterator  28
train iterator  29
train iterator  30
train iterator  31
train iterator  32
train iterator  33
train iterator  34
train iterator  35
train iterator  36
train iterator  37
train iterator  38
train iterator  39
train iterator  40
train iterator  41
train iterator  42
train i

In [14]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss = evaluate(model, test_iterator, criterion, True)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

final tensor 
tensor([[   5,    5,    5,  ...,    8,    8,   18],
        [ 161,  396, 3771,  ...,   16,   28,   32],
        [  34,  221,   28,  ...,   40,   16,   11],
        ...,
        [   3,    3,    3,  ...,    3,    3,    3],
        [   3,    3,    3,  ...,    3,    3,    3],
        [   3,    3,    3,  ...,    3,    3,    3]])
reference 
[['ein', 'brauner', 'hund', 'läuft', 'über', 'den', 'sandstrand', '.', '<eos>']]
candidate 
['ein', 'brauner', 'hund', 'läuft', 'am', 'strand', 'entlang', '.', '<eos>']
reference 
[['ein', 'asiatischer', 'fabrikarbeiter', 'posiert', 'für', 'die', 'kamera', '.', '<eos>']]
candidate 
['ein', 'asiatischer', 'posiert', 'posiert', 'für', 'die', 'kamera', '.', '<eos>']
reference 
[['ein', 'schwarzer', 'junge', 'sitzt', 'i', 'm', 'sand', '.', '<eos>']]
candidate 
['ein', 'fröhlicher', 'junge', 'sitzt', 'i', 'm', 'sand', '.', '<eos>']
reference 
[['ein', 'chinese', 'sitzt', 'und', 'wartet', 'auf', 'kundschaft', '.', '<eos>']]
candidate 
['ein', 'man

/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg


[['drei', 'arbeiter', 'machen', '<unk>', 'auf', 'dem', 'gehsteig', '.', '<eos>']]
candidate 
['drei', 'bauarbeiter', 'sind', 'die', '<unk>', '.', '<eos>']
reference 
[['mann', 'erklettert', 'mauer', 'mit', 'feuer', 'in', 'der', 'hand', '<eos>']]
candidate 
['ein', 'mann', 'hält', 'eine', 'wand', 'mit', 'einer', '<unk>', 'wand', '.', '<eos>']
reference 
[['eine', 'schöne', 'frau', 'spielt', 'auf', 'einer', 'harfe', '.', '<eos>']]
candidate 
['eine', '<unk>', 'frau', 'spielt', 'ein', '<unk>', '.', '<eos>']
reference 
[['leute', ',', 'die', 'vor', 'einem', 'gebäude', 'stehen', '.', '<eos>']]
candidate 
['leute', 'stehen', 'vor', 'einem', 'gebäude', '.', '<eos>']
reference 
[['ein', 'junge', 'steht', 'mit', 'drei', 'mädchen', '.', '<eos>']]
candidate 
['ein', 'junge', 'steht', 'mit', 'drei', 'drei', '.', '<eos>']
reference 
[['zwei', 'männer', 'sitzen', 'in', 'einem', 'restaurant', '.', '<eos>']]
candidate 
['zwei', 'männer', 'sitzen', 'in', 'einem', 'restaurant', '.', '<eos>']
reference 

reference 
[['drei', 'kleine', 'kinder', 'gehen', 'durch', 'einen', 'grasbewachsenen', 'hof', '.', '<eos>']]
candidate 
['drei', 'kleine', 'kinder', 'gehen', 'durch', 'eine', 'wiese', '.', '<eos>']
reference 
[['zwei', 'hunde', '<unk>', 'sich', 'gegenseitig', 'nase', 'an', 'nase', '.', '<eos>']]
candidate 
['zwei', 'hunde', 'kämpfen', 'sich', 'gegenseitig', '.', '<eos>']
reference 
[['eine', 'gruppe', 'von', '<unk>', 'macht', 'wartungsarbeiten', 'an', 'bahngleisen', '.', '<eos>']]
candidate 
['ein', '<unk>', 'ohne', '<unk>', 'findet', '.', '<eos>']
reference 
[['alle', 'personen', 'in', 'der', 'gruppe', 'stehen', 'unter', 'regenschirmen', '.', '<eos>']]
candidate 
['eine', 'gruppe', 'von', 'menschen', 'ist', 'die', 'alle', '.', '<eos>']
reference 
[['asiatische', 'frau', 'trägt', 'einen', 'sonnenhut', 'beim', 'fahrradfahren', '.', '<eos>']]
candidate 
['eine', 'asiatin', 'mit', 'einem', 'fährt', 'fährt', 'fahrrad', '.', '<eos>']
reference 
[['zwei', '<unk>', 'spielen', 'football', 'i',

final tensor 
tensor([[   5,    8,    5,  ...,    5,    5, 1533],
        [  34,  401,    0,  ..., 2537,  999,   56],
        [  11,   16, 1556,  ...,   27,   13,   66],
        ...,
        [   4,    3,    3,  ...,    3,    4,  336],
        [   3,    3,    3,  ...,    3,    3,    4],
        [   3,    3,    3,  ...,    3,  797,    3]])
reference 
[['hund', 'mit', 'schwarzem', 'halsband', 'wälzt', 'sich', 'in', 'staub', 'und', 'trockenen', 'blättern', '.', '<eos>']]
candidate 
['ein', 'hund', 'mit', 'einem', 'halsband', 'und', 'einem', 'schwarzen', 'halsband', 'i', 'm', 'maul', '.', '<eos>']
reference 
[['eine', 'alte', 'frau', 'sitzt', 'an', 'einem', 'webstuhl', 'und', 'stellt', 'stoff', 'her', '.', '<eos>']]
candidate 
['eine', 'alte', 'frau', 'arbeitet', 'an', 'einem', 'webstuhl', 'webstuhl', 'webstuhl', 'webstuhl', '.', '<eos>']
reference 
[['ein', '<unk>', '-', 'biker', 'hilft', 'einem', 'anderen', ',', 'der', 'hingefallen', 'ist', ',', 'auf', '.', '<eos>']]
candidate 
['ein', '<

...

final tensor 
tensor([[  18,    5, 1018,  ...,    5,    5,    5],
        [  49,  183,   76,  ...,   27,   13,   13],
        [ 192,   27,   12,  ...,    7, 1078,   10],
        ...,
        [   3,    3,  831,  ...,    3,    3, 2359],
        [   3,    3,    4,  ...,    3,    3,    4],
        [   3,    3,    3,  ...,    3,    3,    3]])
reference 
[['zwei', 'frauen', 'werden', 'bei', 'einer', 'art', '<unk>', 'an', 'einem', 'baum', '<unk>', '.', '<eos>']]
candidate 
['zwei', 'frauen', 'unterhalten', 'sich', 'in', 'einem', 'baum', 'in', '<unk>', '.', '<eos>']
reference 
[['ein', 'junges', 'mädchen', 'steht', 'neben', 'einer', 'gelben', 'katze', 'auf', 'einer', '<unk>', '.', '<eos>']]
candidate 
['ein', 'junges', 'mädchen', 'steht', 'auf', 'einem', 'gelben', 'neben', 'einem', 'gelben', 'plastik', '.', '<eos>']
reference 
[['fußgänger', 'auf', 'einer', 'straße', 'blicken', 'auf', 'ein', 'kind', 'in', 'einem', 'karton', '.', '<eos>']]
candidate 
['fußgänger', 'gehen', 'auf', 'einer', 'stra

final tensor 
tensor([[ 18,   8,   8,  ...,   5,   5,   5],
        [126,   0,  38,  ...,  13,  68,  13],
        [ 20,  16,  32,  ...,   7,  27,   7],
        ...,
        [  3,   3,   3,  ...,   3,   3,   3],
        [  3,   3,   3,  ...,   3,   3,   3],
        [  3,   3,   3,  ...,   3,   3,   3]])
reference 
[['zwei', 'hunde', 'sind', 'i', 'm', 'schnee', ',', 'einer', 'von', 'ihnen', 'hat', 'etwas', 'i', 'm', 'maul', '.', '<eos>']]
candidate 
['zwei', 'hunde', 'i', 'm', 'schnee', 'hat', 'sich', 'hat', 'sich', 'in', 'den', 'mund', 'und', 'hat', 'etwas', '.', '<eos>']
reference 
[['eine', 'professionell', 'gekleidete', 'frau', 'steht', 'an', 'einem', 'podium', 'und', '<unk>', 'oder', 'diskutiert', 'über', 'etwas', 'wichtiges', '.', '<eos>']]
candidate 
['eine', '<unk>', 'frau', 'in', 'einem', '<unk>', 'steht', 'an', 'einem', '<unk>', 'oder', '<unk>', "'s", '<unk>', "'s", '<unk>', '.', '<eos>']
reference 
[['eine', 'gruppe', 'von', 'männern', 'verwendet', '<unk>', ',', 'um', 'ein', '

final tensor 
tensor([[  5,  18,   5,  ...,   8,   8,   5],
        [ 99,  32,  13,  ...,  16,  16,  13],
        [ 13,  10, 188,  ..., 163,   7,   7],
        ...,
        [  3,   3,   0,  ...,   3,   3,   3],
        [  3,   3,   0,  ...,   3,   3,   3],
        [  3,   3,   4,  ...,   3,   3,   3]])
reference 
[['ein', 'junger', 'mann', 'in', 'einem', 'blauen', 'shirt', 'fährt', 'in', 'einer', 'städtischen', 'gegend', 'über', 'ein', 'geländer', '.', '<eos>']]
candidate 
['ein', 'junger', 'mann', 'in', 'einem', 'blauen', 'hemd', 'fährt', 'auf', 'einem', 'skateboard', 'in', 'einem', 'skateboard', 'in', 'einem', 'skateboard', '.', '<eos>']
reference 
[['zwei', 'männer', ',', 'einer', 'in', 'schwarz', 'und', 'weiß', 'und', 'einer', 'in', 'rot', ',', 'spielen', 'beachvolleyball', '.', '<eos>']]
candidate 
['zwei', 'männer', 'und', 'ein', 'rot', 'in', 'schwarz', 'spielen', 'in', 'weißer', 'kleidung', 'spielen', '.', '<eos>']
reference 
[['ich', 'sehe', 'einen', 'mann', ',', 'der', 'die', 

final tensor 
tensor([[  5,   5,   5,  ...,   5,   5, 225],
        [ 13,  13,  28,  ...,  27,  73,  56],
        [  7,   7,   7,  ...,   7,  28,  11],
        ...,
        [  3,   3,   3,  ...,   3,   3,   3],
        [  3,   3,   3,  ...,   3,   3,   3],
        [  3,   3,   3,  ...,   3,   3,   3]])
reference 
[['ein', 'mann', 'mit', 'kariertem', 'hut', 'in', 'einer', 'schwarzen', 'jacke', 'und', 'einer', 'schwarz', '-', 'weiß', 'gestreiften', 'hose', 'spielt', 'auf', 'einer', 'bühne', 'mit', 'einem', 'sänger', 'und', 'einem', 'weiteren', 'gitarristen', 'i', 'm', 'hintergrund', 'auf', 'einer', 'e', '-', 'gitarre', '.', '<eos>']]
candidate 
['ein', 'mann', 'in', 'einer', 'schwarzen', 'jacke', 'und', 'einer', 'schwarzen', 'jacke', 'spielt', 'auf', 'einer', 'akustikgitarre', 'mit', 'einer', 'akustikgitarre', 'und', 'einer', 'akustikgitarre', 'und', 'einer', 'schwarzen', 'i', 'm', 'hintergrund', '.', '<eos>']
reference 
[['ein', 'mann', 'mit', 'einem', 'grauen', 'shirt', ',', 'blauen', 

count pair  1000
bleu score 
0.1737534378600449
| Test Loss: 3.382 | Test PPL:  29.434 |
